In [4]:
from bs4 import BeautifulSoup as Soup
from datascience import *
import numpy as np

In [5]:
def getAllExtensions(file):
    handler = open(file).read()
    soup = Soup(handler, "lxml")
    return soup.find_all('extension')

In [6]:
master_extensions = getAllExtensions("mastermods.xml")

In [7]:
def getCongMemberExtension(extensions, last_name):
    for extension in extensions:
        ext = str(extension)
        if last_name in ext:
            return extension

In [8]:
def getCongMemberExtensionFromFile(last_name, filename):
    handler = open(filename).read()
    soup = Soup(handler, "lxml")
    extensions = soup.find_all('extension')
    for extension in extensions:
        ext = str(extension)
        if last_name in ext:
            return extension

In [9]:
def getCongMemberTag(congMemberExtension):
    contents = congMemberExtension.contents
    for tag in contents:
        tag_str = str(tag)
        if 'congmember' in tag_str:
            return tag

In [10]:
def getParty(congMemberTag):
    return congMemberTag.attrs['party']
def getType(congMemberTag):
    return congMemberTag.attrs['type']
def getAuthorityId(congMemberTag):
    return congMemberTag.attrs['authorityid']
def getBioGuideId(congMemberTag):
    return congMemberTag.attrs['bioguideid']
def getState(congMemberTag):
    return congMemberTag.attrs['state']
def getCongressId(congMemberTag):
    return congMemberTag.attrs['congress']

In [11]:
def getDistrictTag(congMemberExtension):
    contents = congMemberExtension.contents
    for tag in contents:
        tag_str = str(tag)
        if 'district' in tag_str:
            return tag

In [12]:
def getFirstName(congMemberTag):
    contents = congMemberTag.contents
    name_tags = []
    for tag in contents:
        tag_str = str(tag)
        if 'name' in tag_str:
            name_tags += [tag]
    try:
        first_name = name_tags[1].string.split()[0]
    except:
        first_name = None
    return first_name

Something to consider is completely ignoring delegates altogether??? - avoid collisions and errors that we can't foresee

Also edge case - speaker pro tempore (could be handled by finding his/her name before using these mods methods)

In [13]:
def getCongMemberInfoFromMaster(last_name, mods_filename):
    info = make_array()
    
    try:
        extension = getCongMemberExtension(master_extensions, last_name)
        congMemberTag = getCongMemberTag(extension)
    except:
        return getcongMemberInfoFromLocal(last_name, mods_filename)
    
    congMemType = getType(congMemberTag)
    district = 'N/A'
    if congMemType == 'DELEGATE':
        try:
            info = getcongMemberInfoFromLocal(last_name, mods_filename)
        except:
            info = np.append(info, '')
            info = np.append(info, '')
            info = np.append(info, last_name)
    else:
        if congMemType == 'REPRESENTATIVE':
            try:
                district_tag = getDistrictTag(extension)
                district = district_tag.string
            except:
                district = 'N/A'
        info = np.append(info, getAuthorityId(congMemberTag))
        info = np.append(info, getFirstName(congMemberTag))
        info = np.append(info, last_name)
        info = np.append(info, congMemType)
        info = np.append(info, getParty(congMemberTag))
        info = np.append(info, getState(congMemberTag))
        info = np.append(info, district)
        info = np.append(info, getBioGuideId(congMemberTag))
        info = np.append(info, getCongressId(congMemberTag))
        
        
    return info

In [14]:
getCongMemberInfoFromMaster('Byrne', 'test_mods.xml')

array(['2197', 'Bradley', 'Byrne', 'REPRESENTATIVE', 'R', 'AL', '1',
       'B001289', '114'],
      dtype='<U32')

In [12]:
def getChamber(congMemberTag):
    return congMemberTag.attrs['chamber']

In [13]:
def getCongMemberInfoFromLocal(last_name, mods_filename):
    extension = getCongMemberExtensionFromFile(last_name, mods_filename)
    info = make_array()
    congMemberTag = getCongMemberTag(extension)
    
    congMemChamber = getChamber(congMemberTag)
    congMemType = 'N/A'
    if congMemChamber == 'H':
        congMemType = 'REPRESENTATIVE'
    elif congMemChamber == 'S':
        congMemType = "SENATOR"
    
    district = 'N/A'
    if congMemType == 'REPRESENTATIVE':
        try:
            district_tag = getDistrictTag(extension)
            district = district_tag.string
        except:
            district = 'N/A'
            
    info = np.append(info, getAuthorityId(congMemberTag))
    info = np.append(info, getFirstName(congMemberTag))
    info = np.append(info, last_name)
    info = np.append(info, congMemType)
    info = np.append(info, getParty(congMemberTag))
    info = np.append(info, getState(congMemberTag))
    info = np.append(info, district)
    info = np.append(info, getBioGuideId(congMemberTag))
    info = np.append(info, getCongressId(congMemberTag))
    return info

In [23]:
getCongMemberInfoFromLocal('Ros-Lehtinen', 'test_mods.xml')

array(['985', 'Ileana', 'Ros-Lehtinen', 'REPRESENTATIVE', 'R', 'FL', 'N/A',
       'R000435', '115'],
      dtype='<U32')

In [25]:
test_table = Table().with_columns("speaker_id", make_array(), 
                                "first_name", make_array(), 
                                "last_name", make_array(), 
                                "type" , make_array(),
                                "party", make_array(), 
                                "state", make_array(), 
                                "district", make_array(),
                                "bio_guide_id", make_array(),
                                "congress_id", make_array())

In [ ]:
names = ['Byrne', 'blah']
filenames = ['lskjak', 'kasjdkvjs']